Run these in a notebook cell if you need to install onto your nb env

```python
# 'capture' magic prevents long outputs from spamming your notebook
%%capture pipoutput

# For loading predefined environment variables from files
# Typically used to load sensitive access credentials
%pip install python-dotenv

# Standard python package for interacting with S3 buckets
%pip install boto3

# Interacting with Trino and using Trino with sqlalchemy
%pip install trino[sqlalchemy]

# Pandas and parquet file i/o
%pip install pandas pyarrow fastparquet

# OS-Climate utilities to make data ingest easier
%pip install osc-ingest-tools
```

In [2]:
# use a catalog that is configured for iceberg
ingest_catalog = 'osc_datacommons_iceberg_dev'
ingest_schema = 'iceberg_demo'
ingest_table = 'trino_iceberg_demo'

In [3]:
# load standard credentials and get a database connection
import osc_ingest_trino as osc
osc.load_credentials_dotenv()
engine = osc.attach_trino_engine(verbose=True, catalog=ingest_catalog)

using connect string: trino://erikerlandson@trino-secure-odh-trino.apps.odh-cl1.apps.os-climate.org:443/osc_datacommons_iceberg_dev


/opt/app-root/lib64/python3.8/site-packages/sqlalchemy_trino/dialect.py:259: SAWarning: Dialect trino:rest will not make use of SQL compilation caching as it does not set the 'supports_statement_cache' attribute to ``True``.  This can have significant performance implications including some performance degradations in comparison to prior SQLAlchemy versions.  Dialect maintainers should seek to set this attribute to True after appropriate development and testing for SQLAlchemy 1.4 caching support.   Alternatively, this attribute may be set to False which will disable this warning. (Background on this error at: https://sqlalche.me/e/14/cprf)
  res = connection.execute(sql.text(query)).scalar()


set up some example data in a pandas DF

In [4]:
import pandas as pd
data1 = [
    ['2021Q4', 0.6],
    ['2021Q4', 0.7],
    ['2021Q4', 0.8],
    ['2022Q1', 0.7],
    ['2022Q1', 0.8],
    ['2022Q1', 0.9],
    ['2022Q2', 0.8],
    ['2022Q2', 0.9],
    ['2022Q2', 0.95],
]
df1 = pd.DataFrame(data1, columns = ['quarter', 'reduction'])
df1 = df1.convert_dtypes()
print(df1.info(verbose=True))
df1

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9 entries, 0 to 8
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   quarter    9 non-null      string 
 1   reduction  9 non-null      Float64
dtypes: Float64(1), string(1)
memory usage: 281.0 bytes
None


,quarter,reduction
0,2021Q4,0.6
1,2021Q4,0.7
2,2021Q4,0.8
3,2022Q1,0.7
4,2022Q1,0.8
5,2022Q1,0.9
6,2022Q2,0.8
7,2022Q2,0.9
8,2022Q2,0.95


In [5]:
# make sure schema exists, or table creation below will fail in weird ways
sql = f"""
create schema if not exists {ingest_catalog}.{ingest_schema}
"""
qres = engine.execute(sql)
print(qres.fetchall())

[(True,)]


create a table with a particular data partitioning on our `quarter` column,
and request underlying data to use `ORC` columnar storage format

In [6]:
import osc_ingest_trino as osc
columnschema = osc.create_table_schema_pairs(df1)

tabledef = f"""
create table if not exists {ingest_catalog}.{ingest_schema}.{ingest_table}(
{columnschema}
) with (
    format = 'ORC',
    partitioning = array['quarter']
)
"""
print(tabledef)
qres = engine.execute(tabledef)
print(qres.fetchall())


create table if not exists osc_datacommons_iceberg_dev.iceberg_demo.trino_iceberg_demo(
    quarter varchar,
    reduction double
) with (
    format = 'ORC',
    partitioning = array['quarter']
)

[(True,)]


In [7]:
# Delete all data from our db, so we start with empty table
sql=f"""
delete from {ingest_catalog}.{ingest_schema}.{ingest_table}
"""
qres = engine.execute(sql)
print(qres.fetchall())

[(None,)]


Following is the standard `to_sql` functionality provided by `sqlalchemy-trino`.
This default logic writes the entire pandas dataframe as a single sql `insert`.
Note: on very large data frames this may fail because trino limits the size of sql commands.
Below we will create a custom inserting class to adapt to these limits

In [8]:
# method = 'multi' is important, default will not work
# important to tell it about schema here, and catalog when you create the db connection above
# index = False, unless you declared that as a column when you create the table
# use 'append' mode since we already created the table
df1.to_sql(ingest_table,
           con=engine,
           schema=ingest_schema,
           if_exists='append',
           index=False,
           method='multi')

-1

In [9]:
sql=f"""
select * from {ingest_catalog}.{ingest_schema}.{ingest_table}
"""
pd.read_sql(sql, engine)

,quarter,reduction
0,2022Q1,0.70
1,2022Q1,0.80
2,2022Q1,0.90
3,2022Q2,0.80
4,2022Q2,0.90
5,2022Q2,0.95
6,2021Q4,0.60
7,2021Q4,0.70
8,2021Q4,0.80


pandas also allows you to write your own `callable` insertion method and
provide it for `to_sql` to use.

osc-ingest-tools provides a predefined class `TrinoBatchInsert` that allows you to
break data insertions up into batches,
which is useful if you are inserting large dataframes that may exceed
trino's limits on sql command size

Below we use our custom insertion method.
Note that this method inserts our data in two separate insert commands,
and so it will add TWO snapshots to our iceberg db

In [10]:
df1.to_sql(ingest_table,
           con=engine,
           schema=ingest_schema,
           if_exists='append',
           index=False,
           method=osc.TrinoBatchInsert(batch_size = 5, verbose = True))

inserting 5 records
constructed fully qualified table name as: "iceberg_demo.trino_iceberg_demo"
batch insert result: [(5,)]
inserting 4 records
constructed fully qualified table name as: "iceberg_demo.trino_iceberg_demo"
batch insert result: [(4,)]


we can see that a second copy of our dataframe has been inserted into the table

In [11]:
sql=f"""
select * from {ingest_catalog}.{ingest_schema}.{ingest_table}
"""
pd.read_sql(sql, engine)

,quarter,reduction
0,2021Q4,0.60
1,2021Q4,0.70
2,2021Q4,0.80
3,2021Q4,0.60
4,2021Q4,0.70
5,2021Q4,0.80
6,2022Q1,0.70
7,2022Q1,0.80
8,2022Q2,0.80
9,2022Q2,0.90


iceberg maintains snapshots that capture the state of your database after each operation.
Below is an example of how to look at recent snapshots

In [12]:
sql=f"""
select snapshot_id, committed_at from {ingest_catalog}.{ingest_schema}."{ingest_table}$snapshots"
    order by committed_at desc
    limit 5
"""
qres = engine.execute(sql)
snapshots = qres.fetchall()
snapshots

[(5810673469520150672, '2022-02-25 14:58:40.867 UTC'),
 (4699703140021177177, '2022-02-25 14:58:40.146 UTC'),
 (5147174654935222430, '2022-02-25 14:58:32.208 UTC'),
 (5982042343219821442, '2022-02-25 14:58:28.810 UTC'),
 (1411313338609090893, '2022-02-25 14:56:59.358 UTC')]

And here is the snapshot of the previous state of our db

In [13]:
previous_snapshot = snapshots[1][0]
previous_snapshot

4699703140021177177

You can run your query against a particular snapshot of your db,
as in this example

Notice that the previous snapshot includes the FIRST insertion from our custom method,
but not the second one

In [14]:
sql=f"""
select * from {ingest_catalog}.{ingest_schema}."{ingest_table}@{previous_snapshot}"
"""
pd.read_sql(sql, engine)

,quarter,reduction
0,2021Q4,0.60
1,2021Q4,0.70
2,2021Q4,0.80
3,2022Q1,0.70
4,2022Q1,0.80
5,2022Q1,0.90
6,2022Q1,0.70
7,2022Q1,0.80
8,2022Q2,0.80
9,2022Q2,0.90


You can also roll-back the state of your database to a particular snapshot.

Note - rollback appears to not be robust against dropping a table and re-creating it.
Still investigating what the problem is here.

In [15]:
try:
    sql=f"""
    call {ingest_catalog}.system.rollback_to_snapshot('{ingest_schema}', '{ingest_table}', {previous_snapshot})
    """
    qres = engine.execute(sql)
    print(qres.fetchall())
except Exception as e:
    print(e)

[(True,)]


In [16]:
sql=f"""
select * from {ingest_catalog}.{ingest_schema}.{ingest_table}
"""
pd.read_sql(sql, engine)

,quarter,reduction
0,2022Q1,0.70
1,2022Q1,0.80
2,2022Q1,0.90
3,2022Q1,0.70
4,2021Q4,0.60
5,2022Q1,0.80
6,2022Q2,0.80
7,2022Q2,0.90
8,2022Q2,0.95
9,2021Q4,0.70
